In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

In [2]:
world = data.world_110m.url

In [3]:
eq=pd.read_csv('worldcities.csv')
eq.dropna(inplace=True)
eq.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519
5,Delhi,Delhi,28.6700,77.2300,India,IN,IND,Delhi,admin,15926000.0,1356872604


In [4]:
eq.sort_values('population',axis=0, ascending=False, inplace=True)

In [5]:
eq_map = eq.head(5000)

In [6]:
eq_bars = eq.head(20)

In [7]:
eq_map.shape

(3515, 11)

In [8]:
eq_bars.shape

(20, 11)

In [9]:
# define selection
selector = alt.selection_single(nearest=True, empty='all',encodings=['color'], fields=['city'])

In [10]:
map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#fafafa'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#ffffff', stroke='#706545', strokeWidth=0.5
    ),
    # bubbles
    alt.Chart(eq_map).mark_circle(opacity=0.4).encode(
        size=alt.Size('population:Q', scale=alt.Scale(range=[0, 500])),
        latitude='lat:Q',
        longitude='lng:Q',
        tooltip=['city:N','population:Q'],
        color=alt.condition(selector, alt.value('#336699'), alt.value('lightgray'), legend=None)
    ).add_selection(selector)
)

In [11]:
map = map.project(
    type='naturalEarth1', scale=90, translate=[250, 150]
).properties(height=300, width=500)

In [12]:
bars = alt.Chart(eq_bars).mark_bar().encode(
    x='population:Q',
    y=alt.Y('city:O', sort='-x', axis=alt.Axis(title=' ')),
    color=alt.condition(selector, alt.value('#336699'), 
        alt.value('lightgray'), legend=None
    )
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
)
bars = (bars + text).properties(height=300, width=200).add_selection(selector)

In [13]:
map | bars

alt.HConcatChart(...)

[World Cities Database](https://simplemaps.com/data/world-cities) by [Simple Maps](https://simplemaps.com/) is licensed under [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)